# Exercise 8B - NSGA-II Parallelization

Introduced in Tutorial 8

This exercise will demonstrate how to use pymoo's NSGA-II algorithm with EnergyPlus. We will build off of what we had already learned in Exercise 8A and incorporate the code to run EnergyPlus into it. We will also examine the results and conclude if the NSGA-II algorithm is performing as expected.

### Colour codes

<span style="color:orange;"> Orange text is for emphasis and definitions </span>

<span style="color:lime;"> Green text is for tasks to be completed by the student </span>

<span style="color:dodgerblue;"> Blue text is for Python coding tricks and references </span>

## Load all the necessary Python packages
All packages should work with Conda environment if installed on your machine. Otherwise all necessary packages can be installed in a virtual environment (.venv) in VS Code using: Ctrl+Shift+P > Python: Create Environment > Venv > Python 3.12.x > requirements.txt

<span style="color:orange;"> NOTE: that we are using the *pymoo* package will be used. You may need to install this package using pip.</span>

In [ ]:
import json
import matplotlib.pyplot as plt
from multiprocessing import Pool
import numpy as np
import os
import pandas as pd
from pathlib import Path
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.selection.rnd import RandomSelection
from pymoo.termination import get_termination
from pymoo.optimize import minimize
import time

from src.runEnergyPlus import run_energyPlus
from src.saveResults import saveNSGA2History, saveNSGA2Optimal


## 1. Getting Started

### 1.1 Enter the General Parameters for this run
<span style="color:lime;"> These should be the same from previous exercises. Ensure they are correct.</span>

In [ ]:
# Enter a save name for this run
saveName = "Exercise_8B"

# Enter the path to the directory with your EnergyPlus executable. Enter the full path separated by commas.
ep_dir = Path("c:\\", "EnergyPlusV25-1-0") # Example For Windows

# The weather file to be used for this batch of simulations. This file should be located in the src/weatherData/ directory.
weatherFile = "GBR_ENG_London.Wea.Ctr-St.James.Park.037700_TMYx.2009-2023.epw"

# The baseline file to be used for this simulation. This file should be located in the idfs/ directory
idf_file = "1-storey_baseline.idf"

# The parameters file to be used as part of this simulation
parameters_file = "Exercise 8B.json"

# The maximum number of simulations to be run
n_simulations = None
   

This step creates the full paths for the energyPlus directory, idf file, and weather file and confirms they all exist. Else an exception will be created.

In [ ]:
baseline_idf_path = Path("idfs", idf_file)
weather_file_path = Path("weatherData", weatherFile)
parameters_file_path = Path("simulationParameters", parameters_file)

if not ep_dir.exists():
    raise Exception (f"Could not find energyPlus executable at {ep_dir}.")
if not baseline_idf_path.exists():
    raise Exception (f"Could not find idf_file at {baseline_idf_path}.")
if not weather_file_path.exists():
    raise Exception (f"Could not find weather_file at {weather_file_path}.")
if not parameters_file_path.exists():
    raise Exception (f"Could not find the parameters_file at {parameters_file_path}.")

print (f"The EnergyPlus directory is: {ep_dir}.")
print (f"The baseline idf file is: {baseline_idf_path}.")
print (f"The weather file is: {weather_file_path}.")
print (f"The parameters file is {parameters_file_path}")

### 1.2 Load the parameters file

For this demonstration we will be using the simulationParameters file *Exercise 8B.json*. This file is a trimmed down version which focuses on just five variables all related to windows:
* height
* u_windows
* g_value
* wwr
* fixedShadingDepth

<span style="color:orange;"> Note the limits given for the variables

In [ ]:
with open (parameters_file_path) as f:
    parameters = json.load(f)

print (f"Name                      TYPE        VALUES")
for k,v in parameters.items():
    print (f"{k:<26}{v['type']:<12}{v['values']}")

## 2. Setting Up Pymoo

### 2.1 Problem Class

Similar to the previous exercise, we will define a problem class. This class has the same structure of an \_\_init\_\_ and a \_execute method. Some things to note:
* The paths to files need to passed during initialization
* The ranges *xl* and *xu* come from the simulation parameters files. These are unpacked for each of the variables. The *n_var* also comes from the simulation parameters file.
    * For parameters which are constant, the upper and lower limits will be the same. As a result they will not change during the NSGA-II process.
* The evaluate method is much more complex but contains similar steps to run an EnergyPlus simulation as you have seen before.
    * Note that in this example, I am using the *heatingMax* and *SET > 30°C Degree-Hours* metrics. You will need to edit that line for your coursework

In [ ]:
class Problem_EnergyPlus(Problem): # Note that this needs to be a Problem and not an ElementwiseProblem to run in parallel
    # Initialise the class with these default arguments
    def __init__(self, 
                 ep_dir = None,
                 idf_path = None, 
                 weather_file_path = None,
                 parameters = None,
                 n_obj = 2,
                 n_ieq_constr = 0, 
        ):

        #Inherit attributes and methods from the parent class Problem using super()
        super().__init__()

        # Add the number of objects and inequality constraints as attributes        
        self.n_obj = n_obj
        self.n_ieq_constr = n_ieq_constr

        # Store the information about the EnergyPlus parameters as attributes
        self.parameterNames = list(parameters.keys())
        self.n_var = len(parameters)
        self.xl = np.array([min(x["values"]) for x in parameters.values()])
        self.xu = np.array([max(x["values"]) for x in parameters.values()])


    def _evaluate(self, x, out, *args, **kwargs):
        # prepare the parameters for the multiprocessing pool
        inputs = pd.DataFrame(x, columns = self.parameterNames)
        inputs = inputs.to_dict("records")
        inputs = [(ep_dir, baseline_idf_path, weather_file_path, inputs[i], i) for i in range(len(x))]

        # Determine the number of processors to use
        n_processors = os.cpu_count()

        t0 = time.time()
        with Pool(processes = n_processors) as pool:
            returnValues = pool.starmap(run_energyPlus, inputs)
        t1 = time.time()

        # Un pack the results from the batch simulation
        returnCodes = [i[0] for i in returnValues]
        hourlyResults = [i[1] for i in returnValues]
        resilienceResults = [i[2] for i in returnValues]

        # Check if any simulations had errors
        errors = [x.args for x in returnCodes if x.returncode == 1]
        if len(errors) > 0:
            print (f"The following {len(errors)} simulations had errors:")
            for error in errors:
                print (f"\t{error}")
        else:
            print (f"All simulations completed successfully in {t1 - t0:.4f} s.")

        # Putting both the results dictionaries into a dataframe and concatenating them together
        df = pd.DataFrame(hourlyResults)
        df2 = pd.DataFrame(resilienceResults)

        df = pd.concat([df, df2], axis = 1)

        # Return the results of each objective to the output dictionary 
        out["F"] = np.array([df["heatingMax"], df["SET > 30°C Degree-Hours [°C·hr]"]]).T # NOTE the .T to transpose the array


Create an instance of the problem using the default arguments for the file paths and the parameters to be used.

In [ ]:
problem = Problem_EnergyPlus(
    ep_dir = ep_dir,
    idf_path = baseline_idf_path,
    weather_file_path = weather_file_path, 
    parameters = parameters,
    )

## 2.2 Setting up the Algorithm
Similar to before. Here we are going to use 32 individuals with 16 offspring in each generation.

In [ ]:
algorithm = NSGA2(
    pop_size = 32,
    n_offsprings = 16,
    sampling = FloatRandomSampling(),
    selection = RandomSelection(),
    crossover = SBX(),
    mutation = PM(eta=20),
    eliminate_duplicates = True
)

### 2.3 Setting up the termination criteria. 
For this demonstration I have set it to stop after three minutes. However remember that you can also set it to number of generations.

In [ ]:
termination = get_termination("time", "00:03:00")

### 2.4 Running the algorithm as a minimization program

Because we are using multiprocessing, we need to place this inside a **\_\_name\_\_ == "\_\_main\_\_" block.**

<span style = "color:dodgerblue;">This is to make the code "thread-safe" when spawning new workers by ensuring the scripts only execute when run directly and not when imported in worker subprocesses. If you are interested in more details [Stack Overflow](https://stackoverflow.com/questions/419163/what-does-if-name-main-do)</span>


In [ ]:
if __name__ == "__main__":
    res = minimize(problem,
                algorithm,
                termination,
                seed = 314,
                save_history = True,
                verbose = True,
                )


In [ ]:
# Access the arrays for X and F the same way you would for a class attribute
X = res.X
F = res.F

# Convert each into a dataframe
X = pd.DataFrame(X, columns = parameters.keys())
F = pd.DataFrame(F, columns = ["heatingMax", "SET > 30°C Degree-Hours [°C·hr]"])

print (X)
print (F)

### 2.5 Test your understanding
* <span style="color:lime;">What do you notice about this output that is different to that from Exercise 8A?</span>
* <span style="color:lime;">Does this mean the NSGA-II algorithm failed or succeeded?</span>

### 2.6 Save the Results
As a final step, let's save the results so we can analyze them another time without having to re-run the simulations. I have created two functions to save the history of both X and F, and the optimal solution sets in X and F. Both are located in the file *src/saveResults*

In [ ]:
from src.saveResults import saveNSGA2History, saveNSGA2Optimal

# Save the history of X and F
history_X, history_F = saveNSGA2History(res.history, parameters, saveName)

# Save the final optimal set of X and F
final_generation =  history_F.generation.max()
X, F = saveNSGA2Optimal(X, F, final_generation, saveName)

The files can be reloaded using pd.read_csv()

## 3. Analysis

Some additional analysis of the results of the NSGA_II algorithm.

Optional step - re-load the dataframes

In [ ]:
history_X = pd.read_csv(Path("outputs", "results", f"history_X_{saveName}.csv"))
history_F = pd.read_csv(Path("outputs", "results", f"history_F_{saveName}.csv"))
X = pd.read_csv(Path("outputs", "results", f"optimalSet_X_{saveName}.csv"), index_col = 0)
F = pd.read_csv(Path("outputs", "results", f"optimalSet_F_{saveName}.csv"))

### 3.1 Analyse the objective function space

Plot the objective space from the first to final generation

In [ ]:
# Determine what the final_generation is
final_generation = history_F.generation.max()
generations = [0, 1, 5, final_generation]

# Collect data from the history
F_0 = history_F[history_F.generation == 0]
F_1 = history_F[history_F.generation == 1]
F_5 = history_F[history_F.generation == 5]
F_final = history_F[history_F.generation == final_generation]

# Initialize a plot
fig, ax = plt.subplots(nrows = 2, ncols = 2)

ax[0, 0].scatter(F_0["heatingMax"], F_0["SET > 30°C Degree-Hours [°C·hr]"], s=30, facecolors='none', edgecolors='magenta',)
ax[0, 1].scatter(F_1["heatingMax"], F_1["SET > 30°C Degree-Hours [°C·hr]"], s=30, facecolors='none', edgecolors='magenta',)
ax[1, 0].scatter(F_5["heatingMax"], F_5["SET > 30°C Degree-Hours [°C·hr]"], s=30, facecolors='none', edgecolors='magenta',)
ax[1, 1].scatter(F_final["heatingMax"], F_final["SET > 30°C Degree-Hours [°C·hr]"], s=30, facecolors='none', edgecolors='magenta',)

x_min = history_F["heatingMax"].min()
x_max = history_F["heatingMax"].max()
y_min = history_F["SET > 30°C Degree-Hours [°C·hr]"].min()
y_max = history_F["SET > 30°C Degree-Hours [°C·hr]"].max()

# Set the axis and title for each graph in a for loop
for _ax, _generation in zip(ax.flat, generations):
    _ax.set_xlabel("heatingMax", fontsize = 9)
    _ax.set_ylabel("SET > 30°C Degree-Hours", fontsize = 9)
    _ax.set_xlim(x_min, x_max)
    _ax.set_ylim(y_min, y_max)
    _ax.set_title(f"Generation {_generation}", fontsize=9)

fig.set_figheight(5)
fig.set_figwidth(6)
fig.tight_layout()
plt.show()


<span style="color:lime;">Would you say that we have reached convergence?</span>
<span style="color:lime;">Can you explain why there were so few non-dominated solutions?</span>

### 3.2 Analyse the design space

The following code plots the changes in the design space from generation to generation. The points represent the values for each individual, the solid line represents the mean value of each generation, and the black star represents points which form the optimal design set.

To calculate the mean at each generation, we will use pandas *groupby()* function which groups the dataframe by calues in column, here the generation column

In [ ]:
means = history_X.groupby("generation").mean()
print (means[["height", "u_windows", "g_value", "wwr", "fixedShadingDepth"]])

Make the plots, for each of the five parameters.

In [ ]:
fig, ax = plt.subplots(nrows = 5)

# plot the change in height
ax[0].scatter(history_X["generation"], history_X["height"], s = 15, color = "magenta")
ax[0].scatter(X["generation"], X["height"], s = 75, color = "black", marker = "*")
ax[0].plot(means.index, means["height"], color = "darkMagenta")
ax[0].set_ylabel("height [m]")

# plot the change in u-windows
ax[1].scatter(history_X["generation"], history_X["u_windows"], s = 15, color = "dodgerblue")
ax[1].scatter(X["generation"], X["u_windows"], s = 75, color = "black", marker = "*")
ax[1].plot(means.index, means["u_windows"], color = "blue")
ax[1].set_ylabel("u-value [W/m2K]")

# plot the change in g-value
ax[2].scatter(history_X["generation"], history_X["g_value"], s = 15, color = "lime")
ax[2].scatter(X["generation"], X["g_value"], s = 75, color = "black", marker = "*")
ax[2].plot(means.index, means["g_value"], color = "green")
ax[2].set_ylabel("g-value [--]")

# plot the change in wwr
ax[3].scatter(history_X["generation"], history_X["wwr"], s = 15, color = "orange")
ax[3].scatter(X["generation"], X["wwr"], s = 75, color = "black", marker = "*")
ax[3].plot(means.index, means["wwr"], color = "darkorange")
ax[3].set_ylabel("WWR [--]")

# plot the change in fixed shading depth
ax[4].scatter(history_X["generation"], history_X["fixedShadingDepth"], s = 15, color = "chocolate")
ax[4].scatter(X["generation"], X["fixedShadingDepth"], s = 75, color = "black", marker = "*")
ax[4].plot(means.index, means["fixedShadingDepth"], color = "brown")
ax[4].set_ylabel("Shading depth [m]")
ax[4].set_xlabel("Generation")


fig.set_figheight(10)
fig.set_figwidth(8)
fig.tight_layout()

plt.show()

### 3.2.1 Test your understanding
* <span style="color:lime;">What can you say with regards to convergence for each variable?</span>
* <span style="color:lime;">What design features are primarily being selected for? Is it explainable why these design features are being selected?</span>
* <span style="color:lime;">Does the mean of the final generation, coincide with the non-dominated solutions? Is this OK?</span>


## 4. Tutorial 8 Summary

In this tutorial, you will have learned:
* How to set up an optimization problem in pymoo that uses the NSGA-II algorithm
* How to interpret the results of the NSGA-II algorithm
* How to implement the genetic algorithm with EnergyPlus